<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr1/blob/Kozlov-Alexey-folder/Kozlov%20Alexey/InspectrumClinic_gr1_akozlov_20230515_02week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Рекомендательная система для профпатолога (INSPECTRUM CLINIC)**

Цель: 
Разработать прототип системы, реализующей функцию рекомендательной системы для врача профпатолога.

Алексей Козлов 2023.05.15 - 2023.05.22 (2-я неделя):
обработка данных, создание модели DL-точности первой точности

In [1]:
# Загрузка файлов из облака
import gdown
# Библиотека для работы с массивами данных
import numpy as np
# Библиотека для обработки и анализа данных
import pandas as pd
# Библиотека для построения графиков
import matplotlib.pyplot as plt
# Библиотека для построения графиков
import seaborn as sns

In [2]:
# Загрузка файла датасета
url1 = 'https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx'
url2 = '/content/InspectrumClinic1.xlsx'
gdown.download(url1, url2, quiet=False)

Downloading...
From: https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx
To: /content/InspectrumClinic1.xlsx
100%|██████████| 32.4M/32.4M [00:03<00:00, 10.3MB/s]


'/content/InspectrumClinic1.xlsx'

In [3]:
# Чтение данных (df - DataFrame)
df = pd.read_excel(url2)

---
**Шаг 1. Обработка и анализ данных**
---

In [4]:
# Выведем информацию о заполненности ячеек каждого признака
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982 entries, 0 to 1981
Columns: 4587 entries, ДокументПрохождениеМедосмотра to Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
dtypes: bool(1), datetime64[ns](43), float64(1253), object(3290)
memory usage: 69.3+ MB


**Функции для обработки и представления данных:**

In [42]:
def printRowColCount():
  '''
    Отображаем текущее количества строк и столбцов в датасете
  '''
  print(f'Текущий датасет: число строк: {df.shape[0]}, число колонок: {df.shape[1]}, {df.shape}')

def getColInx(colName):
  '''
    Получение индекса колонки
    вход:
        colName - название колонки
    выход:
        retVal - индекс колонки
  '''
  retVal = df.columns.get_loc(colName)
  return retVal

def getColVals(colInx):
  '''
    Получение списка значений колонки
    вход:
        colInx - индекс колонки
    выход:
        retVal - список значений
  '''
  retVal = df.iloc[:, colInx].tolist()
  return retVal

def getColUniqueVals(colName):
  '''
    Получение списка уникальных значений колонки
    вход:
        colName - название колонки
    выход:
        retVal - список уникальных значений
  '''
  retVal = df[colName].unique().tolist()
  return retVal

def getBoolColNames():
  '''
    Получение списка названий колонок типа bool
  '''
  retVal = list(df.select_dtypes(include=['bool']).columns)
  return retVal

def getDateTimeColNames():
  '''
    Получение списка названий колонок типа datetime64
  '''
  # retVal = list(df.select_dtypes(include=['datetime64[ns]']).columns)
  retVal = list(df.select_dtypes(include=['datetime64']).columns)
  return retVal

def getObjStrColNames():
  '''
    Получение списка названий колонок типа object-string
  '''
  retVal = list(df.select_dtypes(include=['object']).columns)
  return retVal

def patchTextCol(colName):
  '''
    Удаляем в описаниях(текстовых полях) неразрывные пробелы, перевод каретки: \xa0 \n
    вход:
        colName - название колонки
  '''
  # Цикл по всем строкам
  for i in range(df.shape[0]):
    #заменяем фрагменты кодировки ASCII на отсутствие символов
    df.values[i][j] = df.values[i][j].replace("\xa0","")
    df.values[i][j] = df.values[i][j].replace("\n"," ")
#TODO
#    привести к нижнему регистру и удалить пробелы слева и справа

def print1Row(rowInx):
  '''
    Печать одной строки с индексом rowInx
    вход:
        rowInx - индекс строки
  '''
  # Цикл по всем колонкам
#  for j in range(df.shape[1]):
  for j in range(14):
   print(f'{j}: {df.values[rowInx][j]}')

def errRowRemove(colName, errVals):
  '''
    Удаляем строки в датасете
    вход:
        colName - название колонки
        errVals - список запрещенных значений
  '''

  print(f'1>>>>>>>>>>colName: {colName}')
  print(f'2>>>>>>>>>>errVals: {errVals}')

  dropInxList = df[df[colName].isin(errVals)].index
  print(f'3>>>>>>>>>>dropInxList: {dropInxList}')

  print()

def nanRowRemove(colName):
  '''
    Удаляем строки в датасете, которые не имеют значений
    вход:
        colName - название колонки
  '''

  print(f'1>>>>>>>>>>colName: {colName}')



# def drop_rows_by_condition(df, column_name, forbidden_values):
#     indexes_to_drop = df[df[column_name].isin(forbidden_values)].index
#     df = df.drop(index=indexes_to_drop)
#     return df

In [41]:
printRowColCount()
#print1Row(1024)

Текущий датасет: число строк: 1982, число колонок: 4587, (1982, 4587)
0: 59fb827b-616e-11ed-8109-0cc47aab8067
1: abe3549b-4042-11ec-80fa-0cc47aab8067
2: 1989-07-10 00:00:00
3: Мужской
4: Мужской
5: 2022-11-11 00:00:00
6: Периодический
7: Периодический
8: False
9: 00bbe942-2944-11ea-80d7-0cc47aab8067
10: nan
11: Оператор технологических установок 5 разряда
12:  Газоспасатель
13: Годен


**Целевой столбец “ЗаключениеМК”**

In [43]:
targetColName = 'ЗаключениеМК'

strCols = getObjStrColNames()
if targetColName in strCols:
    print(f'"{targetColName}" найдено в списке колонок типа object-string')
else:
    print(f'"{targetColName}" не найдено в списке колонок типа object-string')

# inx = strCols.index(targetColName)
# colInx = getColInx(targetColName)
# print(f'"{targetColName}": индекс в списке: {inx}, индекс в таблице: {colInx}')
# icol = 0
# for colVal in strCols:
#   print(f'col:{icol} |{colVal}|')
#   icol += 1

сolUVals = getColUniqueVals(targetColName)
print(f'сolUVals: {сolUVals}')

# Удаление строк по значению колонки
forbiddenVals = list()
forbiddenVals.append('машинист бульдозера')
forbiddenVals.append('Газоспасатель')
forbiddenVals.append(' Газоспасатель')
errRowRemove(targetColName, forbiddenVals)

print('!!!!!!!!!!!!!!!!')


"ЗаключениеМК" найдено в списке колонок типа object-string
сolUVals: ['Годен', 'ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах', 'НуждаетсяВДообследованииИЛечении', 'ВременноНегоден', 'ОграниченноГоден', nan, 'машинист бульдозера', 'Газоспасатель', ' Газоспасатель']
1>>>>>>>>>>colName: ЗаключениеМК
2>>>>>>>>>>errVals: ['машинист бульдозера', 'Газоспасатель', ' Газоспасатель']
3>>>>>>>>>>dropInxList: Int64Index([843, 851, 1022, 1024], dtype='int64')

!!!!!!!!!!!!!!!!


**Колонка “ПсихОсвидетельствование” типа bool**

In [ ]:
boolCols = getBoolColNames()
icol = 0
for colVal in boolCols:
  print(f'col:{icol} |{colVal}|')
  icol += 1

colInx = getColInx(boolCols[0])
сolVals = getColVals(colInx)
сolUVals = getColUniqueVals(boolCols[0])
print(f'colInx: {colInx}')
#print(f'сolVals: {сolVals}')
print(f'сolUVals: {сolUVals}')

col:0 |ПсихОсвидетельствование|
colInx: 8
сolUVals: [False, True]


Значений типа 'nan', поэтому колонку не исправляем.
Считаю поле важным, поэтому его оставляю для постоения нейросети.

**Колонки типа datetime64**

In [ ]:
dtCols = getDateTimeColNames()
icol = 0
for colVal in dtCols:
  print(f'col:{icol} |{colVal}|')
  icol += 1

col:0 |КлиентДатаРождения|
col:1 |ДатаЗавершенияМедосмотра|
col:2 |ЛабораторныеИсследования1_ДатаПриема|
col:3 |Рентгенология1_ДатаПриема|
col:4 |Рентгенология2_ДатаПриема|
col:5 |ПсихиатрияНаркология1_ДатаПриема|
col:6 |Офтальмология1_ДатаПриема|
col:7 |Оториноларингология1_ДатаПриема|
col:8 |ХирургНеИсп1_ДатаПриема|
col:9 |Гинекология1_ДатаПриема|
col:10 |ФункциональнаяДиагностика1_ДатаПриема|
col:11 |Стоматология1_ДатаПриема|
col:12 |Эндоскопия1_ДатаПриема|
col:13 |Терапия1_ДатаПриема|
col:14 |Дерматовенерология1_ДатаПриема|
col:15 |Неврология1_ДатаПриема|
col:16 |УЗИДиагностика1_ДатаПриема|
col:17 |Хирургия1_ДатаПриема|
col:18 |Хирургия2_ДатаПриема|
col:19 |Терапия2_ДатаПриема|
col:20 |Неврология2_ДатаПриема|
col:21 |Неврология2_ДатаПриема_Значение|
col:22 |Неврология2_ДатаПриема_ЗначениеПредставление|
col:23 |УЗИДиагностика2_ДатаПриема|
col:24 |ЭКГНеИсп1_ДатаПриема|
col:25 |ПроцедурнаяМедсестра1_ДатаПриема|
col:26 |ПсихиатрияНаркология2_ДатаПриема|
col:27 |ПсихиатрияНаркология2_Да

Полей типа МКБ

```
# Выбран кодовый формат
```



In [ ]:





# print('-'*50)
# print(boolCols)
# print(len(boolCols))



# print('='*50)
# for col2 in dtCols:
#   print(f'col:{i2} |{col2}|')
#   i2 += 1


# # Удаляем в описаниях(текстовых полях) неразрывные пробелы, перевод каретки: \xa0 \n
# print('-'*50)
# patchTextField()


Текущий датасет: число строк: 1982, число колонок: 4587, (1982, 4587)
--------------------------------------------------
['ПсихОсвидетельствование']
1
col1: |ПсихОсвидетельствование|
col:0 |КлиентДатаРождения|
col:1 |ДатаЗавершенияМедосмотра|
col:2 |ЛабораторныеИсследования1_ДатаПриема|
col:3 |Рентгенология1_ДатаПриема|
col:4 |Рентгенология2_ДатаПриема|
col:5 |ПсихиатрияНаркология1_ДатаПриема|
col:6 |Офтальмология1_ДатаПриема|
col:7 |Оториноларингология1_ДатаПриема|
col:8 |ХирургНеИсп1_ДатаПриема|
col:9 |Гинекология1_ДатаПриема|
col:10 |ФункциональнаяДиагностика1_ДатаПриема|
col:11 |Стоматология1_ДатаПриема|
col:12 |Эндоскопия1_ДатаПриема|
col:13 |Терапия1_ДатаПриема|
col:14 |Дерматовенерология1_ДатаПриема|
col:15 |Неврология1_ДатаПриема|
col:16 |УЗИДиагностика1_ДатаПриема|
col:17 |Хирургия1_ДатаПриема|
col:18 |Хирургия2_ДатаПриема|
col:19 |Терапия2_ДатаПриема|
col:20 |Неврология2_ДатаПриема|
col:21 |Неврология2_ДатаПриема_Значение|
col:22 |Неврология2_ДатаПриема_ЗначениеПредставление|

In [ ]:
def printDFRowColCount():
  '''
    Отображение текущего количества строк и столбцов в датасете
  '''
  print(f'Текущий датасет: число строк: {df.shape[0]}, число колонок: {df.shape[1]}, {df.shape}')

##############################################
####Убрать кодировку в описаниях \xa0 \n
##############################################
fixed_df = fixed_df.drop(fixed_df.columns[0],axis = 1) # Убираем первый столбец с продублированными индексами. Он нам не нужен

i=0
for j in range(12):  # Для каждого из столбцов таблицы
    for i in range(fixed_df.shape[0]): # По всем резюме
        if type(fixed_df.values[i][j])!=float: # если данные текстовые, то заменяем фрагменты кодировки ASCII на отсутствие символов
            fixed_df.values[i][j] = fixed_df.values[i][j].replace("\xa0","")
            fixed_df.values[i][j] = fixed_df.values[i][j].replace("\n"," ")